In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import hashlib
from ultralytics import solutions
from skimage import exposure
from typing import List, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor

# Constante para o tamanho da janela deslizante
WINDOW_SIZE = 254

class ImageProcessor:
    def __init__(self, model_path: str, window_size: int = WINDOW_SIZE) -> None:
        """
        Inicializa o processador de imagem.

        :param model_path: Caminho para o modelo de contagem de objetos.
        :param window_size: Tamanho da janela para o processamento.
        """
        self.window_size: int = window_size
        self.model_path: str = model_path
        self.image: Optional[np.ndarray] = None
        self.processed_image: Optional[np.ndarray] = None
        self.processed_windows: List[np.ndarray] = []
        self.object_counts: List[int] = []

    def load_image(self, image_path: str) -> np.ndarray:
        """
        Carrega uma imagem do disco e converte de BGR para RGB.

        :param image_path: Caminho para o arquivo de imagem.
        :return: Imagem em formato RGB.
        :raises ValueError: Se a imagem não for carregada.
        """
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Não foi possível carregar a imagem: {image_path}")
        return image

    def normalize_square(self, square: np.ndarray) -> np.ndarray:
        """
        Aumenta o contraste da janela usando ajuste de gama.

        :param square: Janela da imagem.
        :return: Janela com contraste ajustado.
        """
        return exposure.adjust_gamma(square, gamma=1.5)

    def save_processed_window(self, processed_window: np.ndarray, output_folder: str = "processed_windows") -> None:
        """
        Salva uma janela processada com um nome único gerado via hash MD5.

        :param processed_window: Janela processada a ser salva.
        :param output_folder: Pasta onde a janela será salva.
        """
        os.makedirs(output_folder, exist_ok=True)
        contiguous_array = np.ascontiguousarray(processed_window)
        hs = hashlib.md5(contiguous_array.tobytes()).hexdigest()
        file_name = f"{len(self.processed_windows)}_{hs}.png"
        output_path = os.path.join(output_folder, file_name)
        # Converte de RGB para BGR para salvar com cv2.imwrite
        cv2.imwrite(output_path, cv2.cvtColor(processed_window, cv2.COLOR_RGB2BGR))

    def process_window(self, window: np.ndarray) -> np.ndarray:
        """
        Aplica o pipeline de processamento em uma janela.

        :param window: Janela da imagem.
        :return: Janela processada.
        """
        return self.normalize_square(window)

    def count_objects(self, square: np.ndarray) -> Tuple[int, np.ndarray]:
        """
        Conta os objetos na janela utilizando o modelo de contagem.

        :param square: Janela processada.
        :return: Uma tupla contendo o número de objetos e a imagem com as marcações.
        """
        # Região de interesse fixa (pode ser parametrizada se necessário)
        region = [(0, 0), (WINDOW_SIZE, 0), (WINDOW_SIZE, WINDOW_SIZE), (0, WINDOW_SIZE)]
        counter = solutions.ObjectCounter(model=self.model_path, region=region, show=False, boxes=False)
        counted_image = counter.count(square)
        num_objects = len(counter.boxes)
        return num_objects, counted_image

    def process_single_window(self, window: np.ndarray, y: int, x: int, index: int) -> Tuple[int, np.ndarray, int, np.ndarray, int, int]:
        """
        Processa uma única janela: aplica a normalização e a contagem de objetos.

        :param window: A janela a ser processada.
        :param y: Coordenada vertical da janela na imagem.
        :param x: Coordenada horizontal da janela na imagem.
        :param index: Índice da janela (para manter a ordem).
        :return: Tupla contendo o índice, a janela processada, a contagem de objetos,
                 a imagem com a contagem desenhada, e as coordenadas (y, x).
        """
        processed_window = self.process_window(window)
        object_count, counted_image = self.count_objects(processed_window)
        return index, processed_window, object_count, counted_image, y, x

    def apply_window_pipeline(self) -> None:
        """
        Processa a imagem carregada utilizando uma janela deslizante (com preenchimento)
        e utiliza multithreading para processar múltiplos quadrados simultaneamente.
        """
        if self.image is None:
            raise ValueError("Nenhuma imagem foi carregada para processamento.")

        img_height, img_width, channels = self.image.shape

        # Calcula as dimensões com preenchimento
        padded_height = ((img_height + self.window_size - 1) // self.window_size) * self.window_size
        padded_width = ((img_width + self.window_size - 1) // self.window_size) * self.window_size

        # Cria a imagem preenchida com fundo preto
        padded_image = np.zeros((padded_height, padded_width, channels), dtype=self.image.dtype)
        padded_image[:img_height, :img_width, :] = self.image

        # Cria uma cópia para inserir as janelas processadas
        processed_image = np.copy(padded_image)

        # Limpa as listas de janelas e contagens
        self.processed_windows.clear()
        self.object_counts.clear()

        tasks = []
        index = 0

        # Cria um ThreadPool para processar os quadrados simultaneamente
        with ThreadPoolExecutor() as executor:
            for y in range(0, padded_height, self.window_size):
                for x in range(0, padded_width, self.window_size):
                    window = padded_image[y:y + self.window_size, x:x + self.window_size]
                    # Submete a tarefa de processar a janela
                    future = executor.submit(self.process_single_window, window, y, x, index)
                    tasks.append(future)
                    index += 1

        # Coleta os resultados e reordena pela ordem dos índices
        results = [future.result() for future in tasks]
        results.sort(key=lambda r: r[0])

        # Reconstroi as listas e a imagem final a partir dos resultados
        for res in results:
            idx, processed_window, object_count, counted_image, y, x = res
            self.processed_windows.append(processed_window)
            self.object_counts.append(object_count)
            processed_image[y:y + self.window_size, x:x + self.window_size] = counted_image

        # Remove o preenchimento para manter as dimensões originais
        self.processed_image = processed_image[:img_height, :img_width]

    def show_results(self) -> None:
        """
        Exibe a imagem original e a imagem final processada.
        """
        if self.image is None or self.processed_image is None:
            raise ValueError("Imagens não disponíveis para exibição.")
        fig, axes = plt.subplots(2, 1, figsize=(20, 15))
        axes[0].imshow(self.image)
        axes[0].set_title("Imagem Original")
        axes[0].axis("off")
        axes[1].imshow(self.processed_image)
        axes[1].set_title("Imagem Processada Final")
        axes[1].axis("off")
        plt.tight_layout()
        plt.show()

    def show_windows_with_objects(self) -> None:
        """
        Exibe as janelas processadas que contêm objetos.
        """
        indices = [i for i, count in enumerate(self.object_counts) if count > 0]
        if not indices:
            print("Nenhum quadrado contém objetos.")
            return

        n_cols = 2
        n_rows = (len(indices) + n_cols - 1) // n_cols
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 5 * n_rows))
        axes = axes.ravel()
        for i, idx in enumerate(indices):
            window = self.processed_windows[idx]
            axes[i].imshow(window)
            axes[i].set_title(f"Quadrado {idx + 1}\nObjetos: {self.object_counts[idx]}")
            axes[i].axis("off")
        for j in range(i + 1, len(axes)):
            axes[j].axis("off")
        plt.tight_layout()
        plt.show()

    def process_folder(self, folder_path: str) -> None:
        """
        Processa todas as imagens de uma pasta, salva os resultados e gera um relatório.

        :param folder_path: Caminho para a pasta que contém as imagens.
        """
        report: List[Tuple[str, int]] = []
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        for filename in image_files:
            full_path = os.path.join(folder_path, filename)
            try:
                self.image = self.load_image(full_path)
            except ValueError as e:
                print(e)
                continue

            self.apply_window_pipeline()
            total_objects = sum(self.object_counts)
            report.append((filename, total_objects))
            print(f"Processado {filename}: {total_objects} objetos encontrados.")

            # Salva a imagem processada em uma subpasta "processed"
            processed_output_folder = os.path.join(folder_path, "processed")
            os.makedirs(processed_output_folder, exist_ok=True)
            output_image_path = os.path.join(processed_output_folder, f"processed_{filename}")
            cv2.imwrite(output_image_path, cv2.cvtColor(self.processed_image, cv2.COLOR_RGB2BGR))

        # Gera o relatório e salva em um arquivo txt
        report_path = os.path.join(folder_path, "report.txt")
        with open(report_path, "w") as f:
            f.write("+===================================+\n")
            f.write("Filename - Ovos\n")
            f.write(f"Modelo: {os.path.basename(self.model_path)}\n")
            f.write("+===================================+\n")
            for filename, count in report:
                f.write(f"{filename} - {count}\n")
            f.write("+===================================+\n")
        print(f"Relatório salvo em {report_path}")

        # Exibe os 10 quadrados com as maiores contagens (da última imagem processada)
        windows_with_counts = list(enumerate(self.object_counts))
        top_windows = sorted(windows_with_counts, key=lambda x: x[1], reverse=True)[:10]
        n_cols = 2
        n_rows = (len(top_windows) + n_cols - 1) // n_cols
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 5 * n_rows))
        axes = axes.ravel()
        for i, (idx, count) in enumerate(top_windows):
            window = self.processed_windows[idx]
            axes[i].imshow(window)
            axes[i].set_title(f"Quadrado {idx + 1}\nObjetos: {count}")
            axes[i].axis("off")
        for j in range(i + 1, len(axes)):
            axes[j].axis("off")
        plt.tight_layout()
        plt.show()

def main() -> None:
    # Defina os caminhos para o modelo e para a pasta de imagens
    model_path = '/media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/draft-actual/best.pt'
    folder_path = '/media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/base-4'
    
    processor = ImageProcessor(model_path=model_path)
    processor.process_folder(folder_path)
    # Para visualizar os resultados individualmente, descomente as linhas abaixo:
    # processor.show_results()
    # processor.show_windows_with_objects()

if __name__ == "__main__":
    main()
